In [29]:
import pandas as pd
import numpy as np
import datetime 
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from scipy import stats

In [2]:
# 데이터 불러오기
weather=pd.read_csv("risk_data/independent_variable/Weather_risk_table_final.csv", encoding='utf-8-sig')
birds=pd.read_csv("risk_data/independent_variable/birds_risk_table_v2.csv", encoding='utf-8-sig')
distance=pd.read_csv("risk_data/independent_variable/distance_risk_final.csv", encoding='utf-8-sig')

In [3]:
weather.head()

,시도,month,day,avg_temp(C),daily_rain(mm),avg_r_humidity(%),temp_index,rain_index,humidity_index,risk
0,강원영동,1,1,1.2,0.0,33.2,0.0,0.0,0.0,0.0
1,강원영동,1,2,3.2,0.0,34.7,0.0,0.0,0.0,0.0
2,강원영동,1,3,3.4,0.0,35.5,0.0,0.0,0.0,0.0
3,강원영동,1,4,4.2,0.0,39.0,0.0,0.0,0.0,0.0
4,강원영동,1,5,3.2,1.8,43.8,0.0,0.0,0.0,0.0


In [4]:
birds.head()

,Unnamed: 0,month,시도,철새수,발생건수,예측 발생건수,발생건수/예측 발생건수,위험도
0,0,1,강원도,626130,2,9,0.22,0
1,1,1,경기도,1242626,55,16,3.44,3
2,2,1,경상남도,347682,11,6,1.83,1
3,3,1,경상북도,554884,2,8,0.25,0
4,4,1,광주광역시,49915,0,2,0.00,0


In [5]:
distance.head()

,month,dist_range,cnt,cumulative,risk
0,1,0,0,0,10
1,1,10,2,2,10
2,1,20,4,6,10
3,1,30,8,14,10
4,1,40,7,21,10


In [6]:
# 전국 시군 데이터 가져오기
all_sigun=pd.read_csv("risk_data/전국_시군구(완).csv", encoding='utf-8-sig')

In [7]:
all_sigun.head()

,주소,x,y,xy
0,서울특별시 종로구,126.978990,37.573504,"126.978989954189,37.5735042429813"
1,서울특별시 중구,126.997555,37.563808,"126.997555182293,37.5638077703601"
2,서울특별시 용산구,126.990582,37.532431,"126.990582345331,37.5324310391314"
3,서울특별시 성동구,127.036930,37.563427,"127.036930141185,37.563427205337"
4,서울특별시 광진구,127.082385,37.538558,"127.082385189457,37.5385583136667"


In [8]:
all_sigun['시도']=np.nan
all_sigun['시군구']=np.nan
for i in range(len(all_sigun)):
    temp=all_sigun.loc[i,'주소'].split()
    all_sigun.loc[i,'시도']=temp[0]
    all_sigun.loc[i,'시군구']=temp[1]

In [9]:
all_sigun.head()

,주소,x,y,xy,시도,시군구
0,서울특별시 종로구,126.978990,37.573504,"126.978989954189,37.5735042429813",서울특별시,종로구
1,서울특별시 중구,126.997555,37.563808,"126.997555182293,37.5638077703601",서울특별시,중구
2,서울특별시 용산구,126.990582,37.532431,"126.990582345331,37.5324310391314",서울특별시,용산구
3,서울특별시 성동구,127.036930,37.563427,"127.036930141185,37.563427205337",서울특별시,성동구
4,서울특별시 광진구,127.082385,37.538558,"127.082385189457,37.5385583136667",서울특별시,광진구


In [10]:
yeongdong=['강릉시', '동해시', '속초시', '삼척시', '태백시', '고성군', '양양군']
yeongse=['원주시','춘천시','철원군','화천군','양구군','인제군','홍천군','평창군','정선군','영월군','횡성군']

In [11]:
print(set(weather['시도']))

{'경상남도', '경상북도', '강원영서', '전라남도', '전라북도', '서울경기', '충청북도', '강원영동', '충청남도', '제주특별자치도'}


In [12]:
print(set(all_sigun['시도']))

{'경상북도', '경상남도', '강원도', '부산광역시', '대전광역시', '서울특별시', '세종특별자치시', '전라북도', '전라남도', '광주광역시', '충청북도', '충청남도', '제주특별자치도', '인천광역시', '울산광역시', '대구광역시', '경기도'}


In [13]:
# 날씨 시도와 전국 시도 맞추기
w_dic={}
w_dic['인천광역시']='서울경기'
w_dic['대구광역시']='경상북도'
w_dic['부산광역시']='경상남도'
w_dic['울산광역시']='경상남도'
w_dic['서울특별시']='서울경기'
w_dic['대전광역시']='충청남도'
w_dic['경기도']='서울경기'
w_dic['광주광역시']='전라남도'
w_dic['세종특별자치시']='충청남도'
for sido in set(weather['시도']):
    if sido=='강원영동':
        for n in yeongdong:
            w_dic[n]=sido
    elif sido=='강원영서':
        for n in yeongse:
            w_dic[n]=sido
    else:
        w_dic[sido]=sido

In [14]:
print(w_dic)

{'인천광역시': '서울경기', '대구광역시': '경상북도', '부산광역시': '경상남도', '울산광역시': '경상남도', '서울특별시': '서울경기', '대전광역시': '충청남도', '경기도': '서울경기', '광주광역시': '전라남도', '세종특별자치시': '충청남도', '경상남도': '경상남도', '경상북도': '경상북도', '원주시': '강원영서', '춘천시': '강원영서', '철원군': '강원영서', '화천군': '강원영서', '양구군': '강원영서', '인제군': '강원영서', '홍천군': '강원영서', '평창군': '강원영서', '정선군': '강원영서', '영월군': '강원영서', '횡성군': '강원영서', '전라남도': '전라남도', '전라북도': '전라북도', '서울경기': '서울경기', '충청북도': '충청북도', '강릉시': '강원영동', '동해시': '강원영동', '속초시': '강원영동', '삼척시': '강원영동', '태백시': '강원영동', '고성군': '강원영동', '양양군': '강원영동', '충청남도': '충청남도', '제주특별자치도': '제주특별자치도'}


In [15]:
set(distance['month'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [17]:
# 전체 데이터 만들기
cols=['month','day','시도','시군구','철새수','최초발생거리','평균온도','일강수량','상대습도','위험도']
total_df=pd.DataFrame(columns=cols)
total_index_df=pd.DataFrame(columns=cols)
total=[]
total_index=[]
for month in range(1,13):
    start_date='2020-'+str(month)+'-01'
    start_date=datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date=start_date+relativedelta(months=1)-datetime.timedelta(days=1)
    for day in tqdm(range(1,end_date.day+1)):
        for i in range(len(all_sigun)):
            q3=distance.query("month==@month").reset_index(drop=True)
            for j in range(len(q3)):
                temp={}
                index_temp={}
                temp['month']=month
                temp['day']=day
                sido=all_sigun.loc[i,'시도']
                temp['시도']=sido
                sigun=all_sigun.loc[i,'시군구']
                temp['시군구']=sigun
                index_temp['month']=month
                index_temp['day']=day
                index_temp['시도']=all_sigun.loc[i,'시도']
                index_temp['시군구']=all_sigun.loc[i,'시군구']

                q1=birds.query("month==@month and 시도==@sido").reset_index(drop=True)
                if len(q1)==0:
                    temp['철새수']=0
                    index_temp['철새수']=0
                else:
                    temp['철새수']=q1.loc[0,'철새수']
                    index_temp['철새수']=q1.loc[0,'위험도']

                if temp['시도']=='강원도':
                    r_sigun=w_dic[sigun]
                    q2=weather.query("month==@month and day==@day and 시도==@r_sigun").reset_index(drop=True)
                else:
                    r_sido=w_dic[sido]
                    q2=weather.query("month==@month and day==@day and 시도==@r_sido").reset_index(drop=True)
                temp['평균온도']=q2.loc[0,'avg_temp(C)']
                temp['일강수량']=q2.loc[0,'daily_rain(mm)']
                temp['상대습도']=q2.loc[0,'avg_r_humidity(%)']
                index_temp['평균온도']=q2.loc[0,'temp_index']
                index_temp['일강수량']=q2.loc[0,'rain_index']
                index_temp['상대습도']=q2.loc[0,'humidity_index']

                temp['최초발생거리']=q3.loc[j,'dist_range']
                index_temp['최초발생거리']=q3.loc[j,'risk']
                total.append(temp)
                total_index.append(index_temp)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [11:19<00:00, 21.93s/it]


In [18]:
total_df=total_df.append(total)
total_index_df=total_index_df.append(total_index)

In [19]:
total_df.to_csv("risk_data/Train_data_set(risk_x).csv", encoding='utf-8-sig', index=False)

In [20]:
total_index_df.to_csv("risk_data/Train_risk_data_set(risk_x).csv", encoding='utf-8-sig', index=False)

In [2]:
total_df=pd.read_csv("risk_data/Train_data_set(risk_x).csv", encoding='utf-8-sig')

In [3]:
total_index_df=pd.read_csv("risk_data/Train_risk_data_set(risk_x).csv", encoding='utf-8-sig')

In [4]:
total_df.query("시군구=='종로구'").reset_index(drop=True)

,month,day,시도,시군구,철새수,최초발생거리,평균온도,일강수량,상대습도,위험도
0,1,1,서울특별시,종로구,126690,0,-1.8,0.0,64.2,NaN
1,1,1,서울특별시,종로구,126690,10,-1.8,0.0,64.2,NaN
2,1,1,서울특별시,종로구,126690,20,-1.8,0.0,64.2,NaN
3,1,1,서울특별시,종로구,126690,30,-1.8,0.0,64.2,NaN
4,1,1,서울특별시,종로구,126690,40,-1.8,0.0,64.2,NaN
...,...,...,...,...,...,...,...,...,...,...
11822,12,31,서울특별시,종로구,136044,240,-3.2,0.1,62.0,NaN
11823,12,31,서울특별시,종로구,136044,250,-3.2,0.1,62.0,NaN
11824,12,31,서울특별시,종로구,136044,260,-3.2,0.1,62.0,NaN
11825,12,31,서울특별시,종로구,136044,270,-3.2,0.1,62.0,NaN


In [5]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2448189 entries, 0 to 2448188
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   month   int64  
 1   day     int64  
 2   시도      object 
 3   시군구     object 
 4   철새수     int64  
 5   최초발생거리  int64  
 6   평균온도    float64
 7   일강수량    float64
 8   상대습도    float64
 9   위험도     float64
dtypes: float64(4), int64(4), object(2)
memory usage: 186.8+ MB


In [6]:
# 시도, 시군구 범주화 하기
encoded_value, encoded_class = pd.factorize(total_df['시도'])

In [7]:
encoded_value2, encoded_class2 = pd.factorize(total_df['시군구'])

In [8]:
total_df.head()

,month,day,시도,시군구,철새수,최초발생거리,평균온도,일강수량,상대습도,위험도
0,1,1,서울특별시,종로구,126690,0,-1.8,0.0,64.2,NaN
1,1,1,서울특별시,종로구,126690,10,-1.8,0.0,64.2,NaN
2,1,1,서울특별시,종로구,126690,20,-1.8,0.0,64.2,NaN
3,1,1,서울특별시,종로구,126690,30,-1.8,0.0,64.2,NaN
4,1,1,서울특별시,종로구,126690,40,-1.8,0.0,64.2,NaN


In [9]:
sido_sigun=total_df[['시도','시군구']]
sido_sigun['encoded_시도'] = encoded_value
sido_sigun['encoded_시군구'] = encoded_value2
total_df['시도'] = encoded_value
total_df['시군구'] = encoded_value2

C:\Users\user\AppData\Local\Temp/ipykernel_14840/133994541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sido_sigun['encoded_시도'] = encoded_value
C:\Users\user\AppData\Local\Temp/ipykernel_14840/133994541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sido_sigun['encoded_시군구'] = encoded_value2


In [10]:
sido_sigun=sido_sigun.drop_duplicates().reset_index(drop=True)

In [11]:
sido_sigun.to_csv("risk_data/sido_sigun_index.csv", index=False, encoding='utf-8-sig')

In [12]:
sido_sigun.head()

,시도,시군구,encoded_시도,encoded_시군구
0,서울특별시,종로구,0,0
1,서울특별시,중구,0,1
2,서울특별시,용산구,0,2
3,서울특별시,성동구,0,3
4,서울특별시,광진구,0,4


In [13]:
total_df.head()

,month,day,시도,시군구,철새수,최초발생거리,평균온도,일강수량,상대습도,위험도
0,1,1,0,0,126690,0,-1.8,0.0,64.2,NaN
1,1,1,0,0,126690,10,-1.8,0.0,64.2,NaN
2,1,1,0,0,126690,20,-1.8,0.0,64.2,NaN
3,1,1,0,0,126690,30,-1.8,0.0,64.2,NaN
4,1,1,0,0,126690,40,-1.8,0.0,64.2,NaN


In [14]:
# 다중공선성 체크
# 10 이상인 변수도 있고, 엄격하게는 5이상의 변수도 많이 포함되어 있음
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = total_df[list(total_df.columns[:-1])]

vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info['Column'] = X.columns
vif_info.sort_values('VIF', ascending=False)

,VIF,Column
2,63.465309,시도
3,60.927119,시군구
8,22.628293,상대습도
6,7.188542,평균온도
0,5.493268,month
1,4.121860,day
5,3.643663,최초발생거리
4,2.296346,철새수
7,1.813547,일강수량


In [15]:
# 시도, 시군구 상관관계 큼 (다중공선성 10 이상)
# 시도+시군구 번호를 합치기 (다중공선성 제거)
total_df['시도']=total_df['시도']+total_df['시군구']
total_df.drop(['시군구'], axis=1, inplace=True)
total_df=total_df.rename(columns={'시도':'시군구index'})

In [16]:
total_df.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량,상대습도,위험도
0,1,1,0,126690,0,-1.8,0.0,64.2,NaN
1,1,1,0,126690,10,-1.8,0.0,64.2,NaN
2,1,1,0,126690,20,-1.8,0.0,64.2,NaN
3,1,1,0,126690,30,-1.8,0.0,64.2,NaN
4,1,1,0,126690,40,-1.8,0.0,64.2,NaN


In [17]:
# 시군 인덱스 데이터 표도 합치기
sido_sigun['index']=sido_sigun['encoded_시도']+sido_sigun['encoded_시군구']
sido_sigun.to_csv("risk_data/sido_sigun_index.csv", index=False, encoding='utf-8-sig')

In [18]:
# 다중공선성 재확인하기
X = total_df[list(total_df.columns[:-1])]

vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info['Column'] = X.columns
vif_info.sort_values('VIF', ascending=False)

,VIF,Column
7,22.616374,상대습도
5,7.004038,평균온도
0,5.475027,month
1,4.121813,day
2,4.000817,시군구index
4,3.643662,최초발생거리
3,2.032305,철새수
6,1.812828,일강수량


In [19]:
# 상대습도 데이터는 다중공선성이 10이상이며, 독립변수로 역할이 부족할 것이라고 판단 -> 데이터 삭제
total_df.drop(['상대습도'], axis=1, inplace=True)
total_index_df.drop(['상대습도'], axis=1, inplace=True)

In [20]:
# 최종 위험도 부여 (철새수: 0.3, 최초발생거리:0.5, 평균온도:0.14, 일강수량:0.06)
total_index_df['위험도']=total_index_df['철새수']*0.3+total_index_df['최초발생거리']*0.5+total_index_df['평균온도']*0.14+total_index_df['일강수량']*0.06
total_index_df['위험도']=total_index_df['위험도'].map(lambda x : np.round(x,1))

In [21]:
total_df['위험도']=total_index_df['위험도']

In [23]:
total_df.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량,위험도
0,1,1,0,126690,0,-1.8,0.0,7.0
1,1,1,0,126690,10,-1.8,0.0,7.0
2,1,1,0,126690,20,-1.8,0.0,7.0
3,1,1,0,126690,30,-1.8,0.0,7.0
4,1,1,0,126690,40,-1.8,0.0,7.0


In [24]:
total_df.query("시군구index==10").reset_index(drop=True)

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량,위험도
0,1,1,10,126690,0,-1.8,0.0,7.0
1,1,1,10,126690,10,-1.8,0.0,7.0
2,1,1,10,126690,20,-1.8,0.0,7.0
3,1,1,10,126690,30,-1.8,0.0,7.0
4,1,1,10,126690,40,-1.8,0.0,7.0
...,...,...,...,...,...,...,...,...
11822,12,31,10,136044,240,-3.2,0.1,1.4
11823,12,31,10,136044,250,-3.2,0.1,1.4
11824,12,31,10,136044,260,-3.2,0.1,1.4
11825,12,31,10,136044,270,-3.2,0.1,1.4


In [28]:
total_df.isnull().sum()

month       0
day         0
시군구index    0
철새수         0
최초발생거리      0
평균온도        0
일강수량        0
위험도         0
dtype: int64

In [26]:
total_df.to_csv("./risk_data/Train_data_set.csv", encoding='utf-8-sig', index=False)

In [27]:
total_index_df.to_csv("./risk_data/Train_risk_data_set.csv", encoding='utf-8-sig', index=False)